In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from postmodeling.evaluation import (
    get_evaluation,
    get_predictions,
    get_confusion_matrix,
    plot_confusion_matrix,
    get_split_label_df
)
from utils.helpers import get_database_connection, get_label_counts
from utils.constants import LABEL_MAPPING

In [ ]:
db_conn = get_database_connection()

# Prelude: Label size

In [ ]:
labels_dict = {
    'doco': [  # Also used for county='both'
        ['DEATH BY SUICIDE', 'DEATH BY OVERDOSE'],
        ['DEATH BY OVERDOSE', 'SUBSTANCE USE AMBULANCE RUN', 'DOCO DRUG DIAGNOSIS'],

        ['DEATH BY SUICIDE', 'SUICIDE ATTEMPT AMBULANCE RUN', 'SUICIDAL AMBULANCE RUN',
        'DOCO SUICIDE ATTEMPT DIAGNOSIS', 'DOCO SUICIDAL DIAGNOSIS'],

        ['DEATH BY SUICIDE', 'SUICIDE ATTEMPT AMBULANCE RUN', 'DOCO SUICIDE ATTEMPT DIAGNOSIS',
        'DEATH BY OVERDOSE', 'SUBSTANCE USE AMBULANCE RUN', 'DOCO DRUG DIAGNOSIS'],

        ['DEATH BY SUICIDE', 'SUICIDE ATTEMPT AMBULANCE RUN', 'DOCO SUICIDE ATTEMPT DIAGNOSIS',
        'SUICIDAL AMBULANCE RUN', 'DOCO SUICIDAL DIAGNOSIS', 
        'DEATH BY OVERDOSE', 'SUBSTANCE USE AMBULANCE RUN', 'DOCO DRUG DIAGNOSIS',
        'OTHER BEHAVIORAL CRISIS AMBULANCE RUN', 'DOCO OTHER MENTAL CRISIS DIAGNOSIS']
    ],

    'joco': [
        ['DEATH BY SUICIDE', 'DEATH BY OVERDOSE'],
        ['DEATH BY OVERDOSE', 'SUBSTANCE USE AMBULANCE RUN'],
        ['DEATH BY SUICIDE', 'SUICIDE ATTEMPT AMBULANCE RUN', 'SUICIDAL AMBULANCE RUN'],

        ['DEATH BY SUICIDE', 'SUICIDE ATTEMPT AMBULANCE RUN',
        'DEATH BY OVERDOSE', 'SUBSTANCE USE AMBULANCE RUN'],

        ['DEATH BY SUICIDE', 'SUICIDE ATTEMPT AMBULANCE RUN', 'SUICIDAL AMBULANCE RUN',
        'DEATH BY OVERDOSE', 'SUBSTANCE USE AMBULANCE RUN',
        'OTHER BEHAVIORAL CRISIS AMBULANCE RUN']
    ]
}

In [ ]:
def get_label_group(label_names):
    for key, values in LABEL_MAPPING.items():
        for value in values:
            if sorted(value) == sorted(label_names):
                return key

In [ ]:
counties = ['doco', 'joco']

res_dist = []

for county in counties:
    label_names = labels_dict[county]
    for label_name in label_names:
        lg_name = get_label_group(label_name)
        #print(LABEL_MAPPING[lg_name])
        res_dist.append(get_label_counts(db_conn, label_name, lg_name, county=county, distinct_joids=True))

In [ ]:
df_dist = pd.concat(res_dist)

In [ ]:
plt.clf()
plt.figure(figsize=(18, 12))
sns.set(font_scale=1.5)
sns.despine()
sns.set_style('white')
plt.rc("axes.spines", top=False, right=False)

n_colors = df['Label group'].unique().size
palette = sns.color_palette('colorblind', n_colors=n_colors)

p = sns.lineplot(
    data=df_dist.reset_index(), hue='Label group',
    x='As of date', y='Count',
    style='County', hue_order=sorted(LABEL_MAPPING.keys()),
    style_order=['Johnson', 'Douglas'], lw=4, 
    palette=palette
)
title = 'Number of people across label groups and counties'

ylabel = 'Count'
plt.xticks(rotation=45)
p.set(ylabel=ylabel)
plt.title(title, fontsize=24)
legend = plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, ncol=1, frameon=False)

# Overview
This notebook looks at a particular model set across all validation splits and assesses its precision / recall across counties. It also includes crosstabs, feature importance, and fairness audits.

In [ ]:
db_conn = get_database_connection()

# TODO: Write a function to get the model ids from a description of the model (i.e., name and some params)
# These are the random forests ran on Friday 27th July
# They can be swapped for any other model_ids and the code below will then run for those
model_ids = [51, 52, 53, 54, 55] # FeatureRanker on ambulance run the last six months
model_ids = [46, 47, 48, 49, 50] # RandomForest

# Good runs (on expanded labels with all ambulance runs)
model_ids = list(range(2071, 2076)) # High utilizer baseline
model_ids = [2050, 2057, 2059, 2061, 2064] # Feature ranker baseline
model_ids = [2076, 2077, 2078, 2079, 2080] # Huge random forest

## Precision and Recall

In [ ]:
doco_k = 40
joco_k = 75

df = pd.concat(
    get_evaluation(db_conn, id) for id in model_ids
)

In [ ]:
tab = df[['model_id', 'as_of_date', 'county', 'k', 'county_k', 'metric', 'value', 'county_value']]

In [ ]:
def plot_pr_summary(tab, title='', county=False, ax=None, ylim=[0, 0.6], legend=True):
    y = 'county_value' if county else 'value'
    p = sns.lineplot(data=tab, y=y, x='as_of_date', hue='metric', ax=ax, legend=legend);
    p.set(
        title=title,
        ylim=ylim,
        xticks=tab.reset_index().as_of_date
    );
    
def plot_pr_curve(df, title='', county=False, ax=None, xlim=None, ylim=None, legend=True):
    y = 'county_value' if county else 'value'
    p = sns.lineplot(data=df, style='county', x='county_k', y=y, hue='metric', ax=ax, legend=legend)
    p.set(title=title)
    
    if xlim:
        p.set(xlim=xlim)
    if ylim:
        p.set(ylim=ylim)

In [ ]:
# Overall precision / recall at k = 115 across validation splits
overall_tab = tab[tab['k'] == 115].groupby(['model_id', 'as_of_date', 'metric']).mean('value').drop(columns=['k', 'county_k'])

# Precision / recall at k = 70 across validation splits and counties for JoCo
joco_tab = tab[(tab['county'] == 'joco') & (tab['county_k'] == joco_k)].groupby(['model_id', 'as_of_date', 'metric', 'county']).mean('county_value').drop(columns=['k', 'county_k'])

# Precision / recall at k = 40 across validation splits and counties for DoCo
doco_tab = tab[(tab['county'] == 'doco') & (tab['county_k'] == doco_k)].groupby(['model_id', 'as_of_date', 'metric', 'county']).mean('county_value').drop(columns=['k', 'county_k'])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

plot_pr_summary(overall_tab, county=False, title='Across county P/R at k = 115', ax=axs[0])
plot_pr_summary(joco_tab, county=True, title='JoCo P/R at k = 75', ax=axs[1])
plot_pr_summary(doco_tab, county=True, title='DoCo P/R at k = 40', ax=axs[2])

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(18, 10))

xlim=[0, 1000]
ylim=[0, 0.60]
titles = df.reset_index().as_of_date.unique()

k = 0
for i in range(2):
    for j in range(3):
        legend = True if k == 0 else False
        if k < 4:
            plot_pr_curve(df[df['model_id'] == model_ids[k]], county=True, title=titles[k], xlim=xlim, ylim=ylim, ax=axs[i][j], legend=legend)
            k = k + 1

## Crosstabs
In both Johnson and Douglas county we are catching only very, very few people who in fact end up dying by suicide or overdose ...

### Johnson County

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(18, 10))

k = 0
for i in range(2):
    for j in range(3):
        if k < 4:
            cf = get_confusion_matrix(df[df['model_id'] == model_ids[k]], doco_k=None, joco_k=joco_k)
            plt_cf = plot_confusion_matrix(cf)
            axs[i][j].set_title(titles[k])
            plt_cf.plot(ax=axs[i][j])
            k = k + 1

## Douglas County

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(18, 10))

k = 0
for i in range(2):
    for j in range(3):
        if k < 4:
            cf = get_confusion_matrix(df[df['model_id'] == model_ids[k]], doco_k=doco_k, joco_k=None)
            plt_cf = plot_confusion_matrix(cf)
            axs[i][j].set_title(titles[k])
            plt_cf.plot(ax=axs[i][j])
            k = k + 1

## What labels are we predicting?

We read the split labels from the modeling.split_labels table and merge it with the predictions our model makes.

In [ ]:
df_split = get_split_label_df(db_conn, model_ids)

In [ ]:
df_split.sort_values(by='county_k').head(50)

In [ ]:
df_split.value_counts('split_label_name')

In [ ]:
df_split[df_split['split_label_name'].str.contains('DEATH') == True]

In [ ]:
df_split.label_name[0]

In [ ]:
df_split.value_counts('split_label_name')

In [ ]:
# Add predictions to df
# TODO: Test that these counts make sense
df = get_predictions(df)
df_merged = get_split_label_df(db_conn, df)

In [ ]:
df_merged.value_counts('split_label_name')

In [ ]:
# These are the true label counts across prediction = [0, 1]
df_merged.value_counts('split_label_name')

In [ ]:
# These are the true label counts across prediction = 1
# It seems that none of the prediction we make is an actual death ...
df_merged[df_merged['prediction'] == 1].value_counts('split_label_name')

## Feature importance
The most important features are age — by quite a margin — and then the number of mental health diagnoses in Johnson county, followed by ambulance runs. The ranking is fairly stable across validation splits.

In [ ]:
str_model_ids = ','.join([str(id) for id in model_ids])
query = 'select * from results.feature_importance where model_id in({str_model_ids})'.format(str_model_ids=str_model_ids)
df_imp = pd.read_sql(query, db_conn)

In [ ]:
# Feature importance of most recent training / validation split
mean_importance = df_imp[df_imp['model_id'] == model_ids[-1]].groupby(['feature_name']).mean().sort_values('feature_importance', ascending=False)
mean_importance

In [ ]:
sns.barplot(data=mean_importance.reset_index().head(10), y='feature_name', x='feature_importance')
plt.xticks(rotation=90);

## Score distribution

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(18, 10))

# TODO: Make this plots better, the density estimate is a bit misleading
# Histograms are tough because of the huge class imbalance
for i in range(2):
    for j in range(3):
        if (i + j) != 3:
            p = sns.violinplot(data=df[df['model_id'] == model_ids[i+j]], y='score', x='label', ax=axs[i][j])
            p.set(
                title=titles[i+j],
                ylim=[0, 0.2]
            )

In [ ]:
# True labels have around an order of magnitude larger scores
df.groupby(['model_id', 'as_of_date', 'label']).mean('score').drop(columns=['k', 'county_k', 'joid', 'value'])